In [1]:
import pandas as pd 
import numpy as np
import os
from skimage.feature import hog
from skimage import data, exposure
import matplotlib.pyplot as plt
from KNN import knn_predict_instance
#from SVM import kernel_svm_train, kernel_svm_predict
from plotutils import plot_data, plot_surface
from kernelsvm import SVM
import numpy as np


In [2]:
#cwd = os.getcwd() #if need the current directory 
df = pd.read_csv('digit-recognition/digit-recognition/train.csv')
df2 = pd.read_csv('digit-recognition/digit-recognition/test.csv')
#the shape of the dataframe is (38000, 785) meaning there are 38000 samples and 1 label with 784 features

In [16]:
arr = np.array([1,2,3,4,5])
np.where((arr=2) || (arr=1))

SyntaxError: invalid syntax (<ipython-input-16-8d8fde11a52a>, line 2)

In [20]:
#df.iloc[0,:] to access a certain column or row by index
#any other fun commands to put 

#try to interpret the data graphically 

In [3]:
#turn the training dataframe into numpy array 
label_data_train = df.to_numpy()

#column vector for the labels (38000,)
label_train = label_data_train[:,0] 

#( 38000 x 784 matrix for the training data  )
data_train = label_data_train[:, 1:] 
print(label_train.shape)
print(data_train.shape)

#turn testing dataframe into numpy array 
label_data_test = df2.to_numpy()
label_test = label_data_test[:,0]
data_test = label_data_test[:,1:]


(38000,)
(38000, 784)


In [ ]:
#Use sklearn library to try a KNN 
#from sklearn.neighbors import KNeighborsClassifier
#neigh = KNeighborsClassifier(n_neighbors=3)
#neigh.fit(data_train, label_train)
#knn_predictions = neigh.predict(data_test)

In [4]:
#performs feature extraction 
#code taken from https://medium.com/@basu369victor/handwritten-digits-recognition-d3d383431845
#images aren't all same length so don't need to worry about scaling
import matplotlib.pyplot as plt
from skimage.feature import hog
list_hog = []
list_hog1 = []
for feature in data_train:
    fd = hog(feature.reshape((28,28)), orientations=9, pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=False ) #can probably optimize this
    list_hog.append(fd) #4x4 seems to be optimal 
hog_features_train = np.array(list_hog, 'float64')

#on testing data 
for feature in data_test:
    fd = hog(feature.reshape((28,28)), orientations=9, pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=False )
    list_hog1.append(fd)
hog_features_test = np.array(list_hog1, 'float64')


In [ ]:
#hog visualization 
#code taken from https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html
import matplotlib.pyplot as plt
print(hog_features_test.shape)
print(data_test.shape)
image = data_test[5,:].reshape((28,28)) #can do this for multiple images 

fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(1, 1), visualize=True, multichannel=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 20), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [4]:
#Use sklearn library to try SVM model 
#linear appears to work the best with HOG 
from sklearn import svm
from sklearn.multiclass import OneVsOneClassifier
label_train1_2 = np.where(np.logical_or(label_train==2,label_train==1))[0] #either 1 or 2
data_train1_2 = data_train[label_train1_2,:]
labels_1_2 = label_train[label_train1_2]
label_test1_2 = np.where(np.logical_or(label_test==2,label_test==1))[0] #either 1 or 2
data_test1_2 = data_test[label_test1_2,:]
labels_test = label_test[label_test1_2]
clf = OneVsOneClassifier(svm.LinearSVC(C=0.030)) #performs slightyly better on SVM both linear and RBF
#clf = OneVsOneClassifier(svm.SVC(gamma = 0.01, C=.030))
clf.fit(data_train1_2, labels_1_2)
svm_predictions = clf.predict(data_test1_2)
print(svm_predictions.shape)

(876,)


In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsOneClassifier
clf = OneVsOneClassifier(svm.SVC()) #takes a very very long time to run
clf.fit(data_train, label_train)
svm_predictions = clf.predict(data_test)
print(svm_predictions.shape)

In [27]:
#compute accuracy using sklearn 
from sklearn import metrics
print(label_train.shape)
print(label_test.shape)
#print("KNN Accuracy:",metrics.accuracy_score(label_test, knn_predictions))
print("SVM Accuracy:",metrics.accuracy_score(labels_test, svm_predictions))

(38000,)
(4000,)
SVM Accuracy: 0.9840182648401826


In [ ]:
#unewighted KNN with euclidean distance for an instance 
#calculate using euclidean distance 
#would be data_test.shape[0]
knn_predicts = []
for i in range(data_test.shape[0]):
    knn_predicts.append(knn_predict_instance(data_train, label_train, data_test[i,:], 3))


In [ ]:
correct = 0
print(len(knn_predicts))
for i in range(data_test.shape[0]):
    if label_test[i]==knn_predicts[i]:
        correct = correct+1
print(correct/4000)


In [ ]:
#Use sklearn library to try a KNN with HOG 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(hog_features_train, label_train)
knn_predictions = neigh.predict(hog_features_test)

In [5]:
hog_features_train.shape
#one vs one approach
label_train1_2 = label_train[np.logical_or(label_train==1,label_train==2)] #either 1 or 2
hog_features_train1_2 = hog_features_train[np.logical_or(label_train==1,label_train==2),:]
print(hog_features_train1_2.shape)

(7985, 144)


In [ ]:
#for one vs all approach 
label_train1 = np.where(label_train==1, label_train, -1)
label_train2 = np.where(label_train==2, 1, -1)
label_train3 = np.where(label_train==3, 1, -1)
label_train4 = np.where(label_train==4, 1, -1)
label_train5 = np.where(label_train==5, 1, -1)
label_train6 = np.where(label_train==6, 1, -1)
label_train7 = np.where(label_train==7, 1, -1)
label_train8 = np.where(label_train==8, 1, -1)
label_train9 = np.where(label_train==9, 1, -1)

In [25]:
# Run polynomial SVM
#train_data = np.split(data_train, 100)
#train_labels = np.split(label_train, 100)
label_train1_2 = np.where(np.logical_or(label_train==2,label_train==1))[0] #either 1 or 2
data_train1_2 = data_train[label_train1_2,:]
labels_1_2 = label_train[label_train1_2]
#print(data_train)
#print(data_train1_2)
#print(labels_1_2)
#polynomialSVM = SVM(C=0.030)
#polynomialSVM.fit(data_train1_2, labels_1_2)

#print("label_test: {0}".format(label_test))
#predictions = polynomialSVM.predict(data_test)
#print("predictions: {0}".format(predictions))
#correct = np.sum(predictions == label_test)
#print("%d out of %d predictions correct" % (correct, len(predictions)))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[2 1 2 ... 2 1 1]


In [28]:
#Attempt at one vs one approach 
#maybe use np arg where? 
data_train=hog_features_train.T #transpoe so columns are the sample data
print(data_train.shape)
locations1 = np.argwhere(label_train==1)
locations2 = np.argwhere(label_train==2)
locations1 = np.ndarray.flatten(locations1)
locations2 = np.ndarray.flatten(locations2)
data_train[:,np.append(locations1,locations2)]
best_params = {}
best_params['kernel'] = 'linear'
best_params['C'] = 1
label_train[np.logical_or(label_train==1,label_train==2)]
#whaat happens when try to run a multi-class on the single class SVM model? 
#may need to change the labels 
part = data_train[:,np.append(locations1,locations2)]
compare = label_train[np.logical_or(label_train==1,label_train==2)] #somehow still seems to work interesting 
compare = np.where(compare==1, compare, -1) #as model only takes 1, -1
lin_svm_model1 = kernel_svm_train(part, compare, best_params) 


NameError: name 'hog_features_train' is not defined